In [1]:
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.genai import types

print("✅ ADK components imported successfully.")

c:\Users\Sony\Desktop\KaggleXGoogle\.venv\lib\site-packages\google\api_core\_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.0) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


✅ ADK components imported successfully.


In [2]:
research_agent = Agent(
    name ="ResearchAgent",
    model="gemini-2.5-flash-lite",
    instruction="""You are a specialized research agent. Your only job is to use the
    google_search tool to find 2-3 pieces of relevant information on the given topic and present the findings with citations.""",
    tools=[google_search],
    output_key="research_findings"
)
print("✅ Research agent created successfully.")

✅ Research agent created successfully.


In [3]:
summary_agent = Agent(
    name="SummaryAgent",
    model="gemini-2.5-flash-lite",
    instruction="""Read the provided research findings: {research_findings}
Create a concise summary as a bulleted list with 3-5 key points.""",
    output_key="summary"
)

print("✅ Summary agent created successfully.")

✅ Summary agent created successfully.


In [4]:
root_agent = Agent(
    name='root_agent',
    model='gemini-2.5-flash-lite',
    instruction="""You are a research coordinator. Your goal is to answer the user's query by orchestrating a workflow.
1. First, you MUST call the `ResearchAgent` tool to find relevant information on the topic provided by the user.
2. Next, after receiving the research findings, you MUST call the `SummarizerAgent` tool to create a concise summary.
3. Finally, present the final summary clearly to the user as your response.""",
    tools = [
        AgentTool(
            agent=research_agent
        ),
        AgentTool(
            agent=summary_agent,
        )
    ]
)

print("✅ Root agent created successfully.")

✅ Root agent created successfully.


In [5]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug("Provide a summary of the latest advancements in renewable energy technologies.")


 ### Created new session: debug_session_id

User > Provide a summary of the latest advancements in renewable energy technologies.


root_agent > The renewable energy sector is rapidly evolving, with significant progress in both solar and wind power. Solar technology is seeing breakthroughs in efficiency and cost-effectiveness with innovations like perovskite and tandem cells, alongside versatile applications such as bifacial panels and building-integrated photovoltaics. In wind energy, turbines are becoming larger and more efficient, complemented by advancements like direct-drive systems, smart monitoring, and the development of floating offshore and airborne wind energy systems. Furthermore, artificial intelligence is playing a key role in optimizing renewable energy systems and grid integration, while improved energy storage solutions, including advanced battery technologies, are essential for ensuring a reliable and consistent power supply.


In [6]:
response = await runner.run_debug("What are the latest advancements in quantum computing and what do they mean for AI?")


 ### Continue session: debug_session_id

User > What are the latest advancements in quantum computing and what do they mean for AI?


root_agent > Here's a concise summary of the latest advancements in quantum computing and their implications for AI:

*   **Hardware and Error Correction Progress:** Recent advancements (2024-2025) in quantum computing hardware, including improved error correction techniques (e.g., Google's Willow processor, Microsoft's qubit virtualization) and emerging approaches like photonic quantum computing, are making quantum computers more capable and reliable.
*   **Enhanced AI Capabilities and Efficiency:** Quantum computing promises to revolutionize AI by enabling faster, more efficient processing of large datasets, accelerating AI model training, and allowing for more sophisticated models through quantum machine learning.
*   **Revolutionizing Specific AI Fields:** Quantum AI is expected to drive significant breakthroughs in areas like drug discovery, materials science, and natural language processing (NLP), with potential for more efficient language models and advanced chatbots.
*   **Opti

## Sequence of Agent

In [7]:
# Outline Agent: Creates the initial blog post outline.
outline_agent = Agent(
    name="OutlineAgent",
    model="gemini-2.5-flash-lite",
    instruction="""Create a blog outline for the given topic with:
    1. A catchy headline
    2. An introduction hook
    3. 3-5 main sections with 2-3 bullet points for each
    4. A concluding thought""",
    output_key="blog_outline", # The result of this agent will be stored in the session state with this key.
)

print("✅ outline_agent created.")

✅ outline_agent created.


In [8]:
# Writer Agent: Writes the full blog post based on the outline from the previous agent.
writer_agent = Agent(
    name="WriterAgent",
    model="gemini-2.5-flash-lite",
    # The `{blog_outline}` placeholder automatically injects the state value from the previous agent's output.
    instruction="""Following this outline strictly: {blog_outline}
    Write a brief, 200 to 300-word blog post with an engaging and informative tone.""",
    output_key="blog_draft", # The result of this agent will be stored with this key.
)

print("✅ writer_agent created.")

✅ writer_agent created.


In [9]:
# Editor Agent: Edits and polishes the draft from the writer agent.
editor_agent = Agent(
    name="EditorAgent",
    model="gemini-2.5-flash-lite",
    # This agent receives the `{blog_draft}` from the writer agent's output.
    instruction="""Edit this draft: {blog_draft}
    Your task is to polish the text by fixing any grammatical errors, improving the flow and sentence structure, and enhancing overall clarity.""",
    output_key="final_blog", # This is the final output of the entire pipeline.
)

print("✅ editor_agent created.")

✅ editor_agent created.


In [10]:
root_agent = SequentialAgent(
    name="BlogPipline",
    sub_agents=[outline_agent, writer_agent, editor_agent],
)
print("✅ root_agent created as SequentialAgent.")

✅ root_agent created as SequentialAgent.


In [11]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug("Create a blog post about the benefits of meditation for mental health.")


 ### Created new session: debug_session_id

User > Create a blog post about the benefits of meditation for mental health.
OutlineAgent > ## Blog Outline:

**Headline:** Unlock Your Inner Peace: How Meditation Can Revolutionize Your Mental Health

**Introduction Hook:** Feeling overwhelmed by the constant buzz of modern life? Juggling deadlines, social pressures, and an endless to-do list can leave your mind feeling like a chaotic storm. But what if there was a simple, accessible tool that could bring calm, clarity, and resilience to your mental landscape? Discover the transformative power of meditation and how it can be your secret weapon for a healthier, happier mind.

### Main Section 1: Taming the Anxious Beast: Reducing Stress and Anxiety

*   **Calming the Nervous System:** Meditation activates the parasympathetic nervous system, counteracting the "fight or flight" response and promoting a state of relaxation.
*   **Shifting Your Perspective:** Regular practice helps you observe 

### Parell Agent

In [12]:
# Tech Researcher: Focuses on AI and ML trends.
tech_researcher = Agent(
    name="TechResearcher",
    model="gemini-2.5-flash-lite",
    instruction="""Research the latest AI/ML trends. Include 3 key developments,
the main companies involved, and the potential impact. Keep the report very concise (100 words).""",
    tools=[google_search],
    output_key="tech_research", # The result of this agent will be stored in the session state with this key.
)

print("✅ tech_researcher created.")

✅ tech_researcher created.


In [ ]:
# Health Researcher: Focuses on medical breakthroughs.
health_researcher = Agent(
    name="HealthResearcher",
    model="gemini-2.5-flash-lite",
    instruction="""Research recent medical breakthroughs. Include 3 significant advances,
their practical applications, and estimated timelines. Keep the report concise (100 words).""",
    tools=[google_search],
    output_key="health_research", # The result will be stored with this key.
)

print("✅ health_researcher created.")

✅ health_researcher created.


In [14]:
# Finance Researcher: Focuses on fintech trends.
finance_researcher = Agent(
    name="FinanceResearcher",
    model="gemini-2.5-flash-lite",
    instruction="""Research current fintech trends. Include 3 key trends,
their market implications, and the future outlook. Keep the report concise (100 words).""",
    tools=[google_search],
    output_key="finance_research", # The result will be stored with this key.
)

print("✅ finance_researcher created.")

✅ finance_researcher created.


In [15]:
# The AggregatorAgent runs *after* the parallel step to synthesize the results.
aggregator_agent = Agent(
    name="AggregatorAgent",
    model="gemini-2.5-flash-lite",
    # It uses placeholders to inject the outputs from the parallel agents, which are now in the session state.
    instruction="""Combine these three research findings into a single executive summary:

    **Technology Trends:**
    {tech_research}
    
    **Health Breakthroughs:**
    {health_research}
    
    **Finance Innovations:**
    {finance_research}
    
    Your summary should highlight common themes, surprising connections, and the most important key takeaways from all three reports. The final summary should be around 200 words.""",
    output_key="executive_summary", # This will be the final output of the entire system.
)

print("✅ aggregator_agent created.")

✅ aggregator_agent created.


In [18]:
parallel_research_agent = ParallelAgent(
    name="ParallelResearchTeam",
    sub_agents=[tech_researcher, health_researcher, finance_researcher],
)
root_agent = SequentialAgent(
    name="ResearchPipeline",
    sub_agents=[parallel_research_agent, aggregator_agent],
)
print("✅ root_agent created as SequentialAgent.")

ValidationError: 1 validation error for ParallelAgent
  Value error, Agent `TechResearcher` already has a parent agent, current parent: `ParallelResearchTeam`, trying to add: `ParallelResearchTeam` [type=value_error, input_value={'name': 'ParallelResearc...l_error_callback=None)]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/value_error

In [20]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug("Run the daily executive briefing on Tech, Health, and Finance")


 ### Created new session: debug_session_id

User > Run the daily executive briefing on Tech, Health, and Finance
TechResearcher > **AI/ML Trends Shaping Technology, Health, and Finance**

**Key Developments:**

1.  **Generative AI and Multimodal AI:** Advancements in models like GPT-4 and Google's Imagen allow for the creation of diverse content (text, images, video) and the processing of multiple data types (text, audio, visuals).
2.  **AI in Healthcare:** AI is revolutionizing diagnostics, drug development, personalized treatment plans, and telemedicine, improving patient care and accessibility. Companies like Atomwise and Deep Genomics are leveraging AI for faster drug discovery.
3.  **Fintech Innovations (AI/ML-driven):** AI and machine learning are enhancing fraud detection, risk assessment, customer service via chatbots, and personalized investment strategies.

**Main Companies Involved:**

*   **AI/ML:** OpenAI, Google, Microsoft, IBM, AWS, Nvidia
*   **Healthcare:** Atomwise, 

### Loop Workflows

In [21]:
# This agent runs ONCE at the beginning to create the first draft.
initial_writer_agent = Agent(
    name="InitialWriterAgent",
    model="gemini-2.5-flash-lite",
    instruction="""Based on the user's prompt, write the first draft of a short story (around 100-150 words).
    Output only the story text, with no introduction or explanation.""",
    output_key="current_story", # Stores the first draft in the state.
)

print("✅ initial_writer_agent created.")

✅ initial_writer_agent created.


In [22]:
# This agent's only job is to provide feedback or the approval signal. It has no tools.
critic_agent = Agent(
    name="CriticAgent",
    model="gemini-2.5-flash-lite",
    instruction="""You are a constructive story critic. Review the story provided below.
    Story: {current_story}
    
    Evaluate the story's plot, characters, and pacing.
    - If the story is well-written and complete, you MUST respond with the exact phrase: "APPROVED"
    - Otherwise, provide 2-3 specific, actionable suggestions for improvement.""",
    output_key="critique", # Stores the feedback in the state.
)

print("✅ critic_agent created.")

✅ critic_agent created.


In [23]:
# This is the function that the RefinerAgent will call to exit the loop.
def exit_loop():
    """Call this function ONLY when the critique is 'APPROVED', indicating the story is finished and no more changes are needed."""
    return {"status": "approved", "message": "Story approved. Exiting refinement loop."}

print("✅ exit_loop function created.")

✅ exit_loop function created.


In [24]:
refiner_agent = Agent(
    name="RefinerAgent",
    model="gemini-2.5-flash-lite",
    instruction="""You are a story refiner. You have a story draft and critique.
    
    Story Draft: {current_story}
    Critique: {critique}
    
    Your task is to analyze the critique.
    - IF the critique is EXACTLY "APPROVED", you MUST call the `exit_loop` function and nothing else.
    - OTHERWISE, rewrite the story draft to fully incorporate the feedback from the critique.""",
    output_key="current_story", # Updates the story draft in the state.
    tools=[FunctionTool(exit_loop)]

)
print("✅ refiner_agent created.")

✅ refiner_agent created.


In [25]:
story_refinement_loop = LoopAgent(
    name="StoryRefinementLoop",
    sub_agents=[critic_agent, refiner_agent],
    max_iterations=2
)

root_agent = SequentialAgent(
    name="StoryPipeline",
    sub_agents=[initial_writer_agent, story_refinement_loop],
)
print("✅ root_agent created as SequentialAgent.")

✅ root_agent created as SequentialAgent.


In [26]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug("Write a short story about a lighthouse keeper who discovers a mysterious, glowing map.")


 ### Created new session: debug_session_id

User > Write a short story about a lighthouse keeper who discovers a mysterious, glowing map.
InitialWriterAgent > Elias polished the lantern glass, its brass gleam a familiar comfort. For forty years, his world had been the rhythmic sweep of light, the crash of waves, and the salty kiss of the sea air. Tonight, however, a different luminescence caught his eye. Tucked beneath a loose stone near the base of the tower was a rolled parchment. It unrolled with a dry crackle, revealing not paper, but something akin to hardened kelp. And it glowed.

Faint, ethereal lines pulsed across its surface, forming shapes Elias had never seen before – not constellations, not coastlines. A shimmering X marked a spot far out in the inky blackness of the ocean. He traced a glowing curve with a calloused finger, a tremor of something akin to excitement, a feeling long dormant, stirring within him. The map hummed, a silent song of a world beyond his lonely vigil